# About the variance of LOO estimate

Let $X_i = \widehat{\text{elpd}}_{loo,i}$ and consider
\begin{align}
X &= [X_1, X_2, \dots, X_n]^T \\
C := \operatorname{Cov}(X) &= 
\begin{bmatrix}
    \sigma^2 & \gamma   &  \cdots \\
    \gamma   & \sigma^2 &  \\
    \vdots    &          & \ddots
\end{bmatrix} \\
m := \operatorname{E}(X) &= [\mu, \mu, \dots, \mu]^T
\end{align}

We are interested
$$
\theta := \operatorname{Var} \left( \sum_{i=1}^n X_i \right) = \sum_{i,j} C_{i,j} = n \sigma^2 + (n^2 - n) \gamma.
$$
We have one draw of $X$.
Simple naive estimate for $\theta$:
\begin{equation}
\hat{\theta}_\text{naive} = n \operatorname{V}_{i=1}^n X_i
   = n \; \frac{1}{n-1}\sum_{i=1}^n \left(X_i - \hat{\mu} \right)^2
\end{equation}
Currently we are considering a better estimate:
\begin{equation}
\hat{\theta}_\text{better} = n \operatorname{V}_{i=1}^n X_i + (n^2 - n) \hat{\gamma}
\end{equation}
where we are trying to estimate and add the missing contribution from $(n^2 - n) \gamma$.

However,
\begin{equation}
\operatorname{E}\left[ \operatorname{V}_{i=1}^n X_i \right] = \sigma^2 - \gamma \neq \sigma^2
\end{equation}
(see the calculations in appendix and the experiment).
Thus, even if we would estimate the missing term $(n^2 - n) \gamma$ without bias or we would know it, our better estimator $\hat{\theta}_\text{better}$ is biased, $\operatorname{E}\left[ \hat{\theta}_\text{better} \right] \neq \theta$.

Instead we might want to define our better estimator as:
$$
\hat{\theta}_\text{better2} = n \operatorname{V}_{i=1}^n X_i + n^2 \hat{\gamma}.
$$

The following code shows an example.

In [1]:
library(matrixStats)
library(MASS)

In [2]:
# configurations

n = 5
n_trial = 10000

# true distribution parameters
mu_true = 2.0
sigma2_true = 1.5
gamma_true = 0.2

In [3]:
# calculate true theta
theta_true = n*sigma2_true + (n^2 - n)*gamma_true

In [4]:
# form true mean and cov for X
m_vec = rep(mu_true, n)
C_mat = (sigma2_true - gamma_true)*diag(n) + gamma_true*array(1, c(n, n))

In [5]:
# draw n_trial random samples
Xs = mvrnorm(n_trial, m_vec, C_mat)

## Inspect different estimators

In [6]:
# A estimator
As = rowVars(Xs)

In [7]:
cat('A estimator expectation:\n')
cat(sprintf('    %g\n', mean(As)))
cat('theoretical:\n')
cat(sprintf('    %g\n', sigma2_true - gamma_true))

A estimator expectation:
    1.29732
theoretical:
    1.3


In [8]:
# B estimator
Bs = array(0, c(n_trial))
num_of_pairs = choose(n, 2)
for (trial_i in 1:n_trial) {
    X = Xs[trial_i,]
    sum_of_product = 0.0
    for (i in 1:(n-1)){
        for (j in (i+1):n){
            sum_of_product = sum_of_product + X[i]*X[j]
        }
    }
    sum_of_product = sum_of_product / num_of_pairs
    Bs[trial_i] = sum_of_product
}

In [9]:
cat('B estimator expectation:\n')
cat(sprintf('    %g\n', mean(Bs)))
cat('theoretical:\n')
cat(sprintf('    %g\n', gamma_true + mu_true^2))

B estimator expectation:
    4.15743
theoretical:
    4.2


In [10]:
# C estimator
Cs = rowMeans(Xs^2)

In [11]:
cat('C estimator expectation:\n')
cat(sprintf('    %g\n', mean(Cs)))
cat('theoretical:\n')
cat(sprintf('    %g\n', sigma2_true + mu_true^2))

C estimator expectation:
    5.45475
theoretical:
    5.5


In [12]:
# D estimator
Ds = rowMeans(Xs)^2

In [13]:
cat('D estimator expectation:\n')
cat(sprintf('    %g\n', mean(Ds)))
cat('theoretical:\n')
cat(sprintf('    %g\n', sigma2_true/n + ((n-1)/n)*gamma_true + mu_true^2))

D estimator expectation:
    4.41689
theoretical:
    4.46


## True variance for the sum

In [14]:
# sample variance of the sum
sample_var_sum = var(rowSums(Xs))

In [15]:
cat('sample variance of the sum:\n')
cat(sprintf('    %g\n', sample_var_sum))
cat('theoretical:\n')
cat(sprintf('    %g\n', n*sigma2_true + (n^2-n)*gamma_true))

sample variance of the sum:
    11.3462
theoretical:
    11.5


In [16]:
# the naive estimate for theta
theta_naive_s = n*rowVars(Xs)

In [17]:
# compare the naive estimate to the true theta
cat('expectation of the naive estimate for theta:\n')
cat(sprintf('    %g\n', mean(theta_naive_s)))
cat('true theta:\n')
cat(sprintf('    %g\n', n*sigma2_true + (n^2-n)*gamma_true))

expectation of the naive estimate for theta:
    6.48658
true theta:
    11.5



### Appendix

\begin{align}
\operatorname{E}\left( X_i X_j \right)
    &= \operatorname{E}\left( X_i \right) \operatorname{E}\left( X_j \right)
        + \operatorname{Cov}\left(X_i, X_j \right)
        \qquad \text{from the definition of covariance}\\
    &= \begin{cases}
        \mu^2 + \sigma^2, \qquad \text{when} \quad i = j, \\
        \mu^2 + \gamma, \qquad \text{when} \quad i \neq j. \\
    \end{cases}
\end{align}

#### variance for each trial

\begin{align}
\qquad V := \operatorname{V}_{i=1}^n X_i
    &= \frac{1}{n-1}\sum_{i=1}^n \left(X_i - \hat{\mu} \right)^2 \\
    &= \frac{1}{n-1}\sum_{i=1}^n \left(X_i - \frac{1}{n}\sum_{i=1}^n X_i \right)^2\\
    &= \frac{1}{n-1}\sum_{i=1}^n \left(
        \frac{n-2}{n}X_i^2
        -\frac{2}{n} \sum_{j \neq i} X_i X_j
        +\frac{1}{n^2} \sum_{j=1}^{n} \sum_{k \neq j} X_j X_k
        +\frac{1}{n^2} \sum_{j=1}^{n} X_j^2
    \right)^2 \\
\operatorname{E}(V)
    &= \frac{1}{n-1}\sum_{i=1}^n \Biggl(
        \frac{n-2}{n} \operatorname{E}\left( X_i^2 \right)
        -\frac{2}{n} \sum_{j \neq i} \operatorname{E}\left( X_i X_j \right)
     \\ & \qquad \qquad \qquad
        +\frac{1}{n^2} \sum_{j=1}^{n} \sum_{k \neq j} \operatorname{E}\left( X_j X_k \right)
        +\frac{1}{n^2} \sum_{j=1}^{n} \operatorname{E}\left( X_j^2 \right)
    \Biggr)^2 \\
    &= \frac{1}{n-1}\sum_{i=1}^n \Biggl(
        \frac{n-2}{n} \left( \mu^2 + \sigma^2 \right)
        -\frac{2}{n} (n-1) \left( \mu^2 + \gamma \right)
     \\ & \qquad \qquad \qquad
        +\frac{1}{n^2} n (n-1) \left( \mu^2 + \gamma \right)
        +\frac{1}{n^2} n \left( \mu^2 + \sigma^2 \right)
    \Biggr)^2 \\
    & = \frac{1}{n-1}\sum_{i=1}^n \left(
        \frac{n-1}{n} \sigma^2
        -\frac{n-1}{n} \gamma
    \right)^2 \\
    &= \sigma^2 - \gamma \neq \sigma^2
\end{align}